L'explorateur de corpus est un script modifiable pour enrichir un corpus structuré de contenu texte. 
La forme ipynb - ou python notebook signifi que le code est organisé selon des blocs qui peuvent être roulé tous ou en partie selon les objectifs d'analyse ou l'utilisation du texte.
Il contient des fonctions de nettoyage du texte,  de regroupement (clustering), 

In [ ]:

# environnement python 3.8.10

# Le premier bloc doit etre roulé en mode administrateur
# Installation des librairies et packages nécessaire au script
# Cette opération ne devrait être exécutée qu'une seule fois (ou à même le terminal) 
! pip install --upgrade pip
! pip install spacy pandas matplotlib sklearn openpyxl wget



# # Téléchargement du modèle camemBERT pour le Français basé sur l'architecture roBerta
# ! python -m wget https://dl.fbaipublicfiles.com/fairseq/models/camembert-large.tar.gz

# # Décompression du fichier tar
# ! python -m tarfile -e camembert-large.tar.gz

# Intallation des packages
! pip install flair fairseq sentencepiece transformers bitarray omegaconf
# md pour modèle médium, sm pour modèle small et lg pour modèle large
! python -m spacy download fr_core_news_sm  
! python -m spacy download en_core_web_sm  

! pip install nltk

In [ ]:
#import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import string as st
import time
import spacy
import re
import torch
import itertools

In [ ]:
#Import des morceaux de packages 

from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from spacy.lang.fr import French
from spacy.lang.en import English
from torch.nn.utils.rnn import pad_sequence
from transformers import CamembertTokenizer
from flair.data import Sentence
from flair.models import SequenceTagger
from fairseq.models.roberta import CamembertModel
from nltk.stem.snowball import FrenchStemmer
from nltk.stem.snowball import EnglishStemmer

In [ ]:
# Variables globales utilisées pour modifier les explorations des champs de texte dans les corpus


# LISTE_COLONNE correspond à la liste des noms des colonnes sélectionnées pour les analyses, par défaut le nom des colonnes sont les mêmes que le fichier importé
# Il est suggéré de modifié à la source s'il y a confusion. Ces nom n'ont pas à être abrévié mais ne doivent pas contenir de caractères spéciaux comme le / et les "
LISTE_COLONNE = ['T1','U1']
# Dans quelle langue est le corpus ? EN pour English et FR pour Français. Si le corpus est multilingue, il doit être séparé selon la langue avant le traitement. 
# Pour ajouter de nouvelles langues, il faut un modèle de langue pour chacune (modèle, stop words, stemmer...) et ajouter aux conditions.
LANGUE_CORPUS = 'EN'

# PATH_CORPUS contient le chemin et le nom du fichier original 
PATH_CORPUS = "corpus.xlsx"

# chargement d'un dictionnaire personnalisé d'ensembles de synonymes
PATH_SYNSET = "synset.xlsx"

# La liste des termes
termes_temp = ['chef', 'chefs', 'art', 'arts', 'creativ', 'fine', 'culina', 'quality', 'spécialité', 
'gourmet', 'gourmand', 'shack', 'fried', 'fastfood', 'fast-food', 'fast food', 'tradition', 'stand', 'cantine mobile', 
'cantine', 'popote roulante', 'usine', 'travailleur', 'cuisine', 'economic', 'restaurants', 'meals', 'dishes', 'vegan', 'festival', 'foodies', 'restaurateur', 'meal']
# La liste des thèmes 

aliments_temp = ['castor',  'taco', 'burger', 'hot-dog', 'poutine', 'patates', 'potato', 'fries', 'burrito', 'churros', 
'frite', 'friture', 'cheese','lentil', 'beef', 'maple', 'salad', 'pork', 'salmon', 'tartar', 'meat', 'caviar', 'fish', 'cookies', 'seafood','strawberry', 'gaufre']


In [ ]:

# chargement de la fonction de racinisation selon la langue
print(LANGUE_CORPUS)
if LANGUE_CORPUS == 'FR':
    STEMMER = FrenchStemmer()
elif LANGUE_CORPUS == 'EN':
    STEMMER = EnglishStemmer()
else :
    print('erreur, il y a un problème avec la langue du corpus')


In [ ]:

ALIMENTS = []
TERMES = []

for w in aliments_temp:
    ALIMENTS.append(STEMMER.stem(w))
print("Liste aliment racines:", ALIMENTS)

for w in termes_temp:
    TERMES.append(STEMMER.stem(w))
print("Liste termes racines:", TERMES)


display(ALIMENTS)
display(TERMES)


In [ ]:
def get_cat_tree(path):
    df1 = pd.read_excel(path, usecols=col) 
    display(df1)

    return df1

In [ ]:

def get_dataset(path, col = None):
#la fonction qui importe et créé le dataset à partir du corpus 
    df1 = pd.read_excel(path, usecols=col) 
    #par défaut, toutes le colonnes de la table sont chargé en df1

    display(df1)
   
    return df1


In [ ]:

def merge_col(df, colSelect):
# la fonction qui fusionne des colonnes selon une liste sélectionnée dans la variable globale LISTE_COLONNE.


    df_merged = pd.DataFrame(columns=['raw', 'line'])
    df_merged['raw'] = df[colSelect].apply(
        lambda row: (" ".join(row.values.astype(str))), axis=1
    )
    # Ajouté : df_merged[raw] qui contient la fusion des colonnes sélectionnées sans le retrait des majuscules, le df retourné contient 2 col.
    df_merged['line'] = df[colSelect].apply(
        lambda row: (" ".join(row.values.astype(str)).lower()), axis=1
    )

    display(df_merged)
    return df_merged

In [ ]:
def remove_punct(text):
# retire la ponctuation du texte
    translator = str.maketrans(st.punctuation, ' '*len(st.punctuation))
    text = text.translate(translator)
    text = text.replace("d'"," ").replace("l'"," ").replace("un"," ").replace("une"," ").replace("’"," ").replace("«"," ").replace("»"," ")
    return text

In [ ]:
def tokenize(text):
# sépare les mots
    text = re.split('\s+' ,text)
    return [x.lower() for x in text]

In [ ]:
def remove_small_words(tokens):
# selon le type de traitement et d'analyse, il peut être utile de déterminer un nombre minimum de lettre pour que le mot soit considéré.
# pour tout garder, remplacer le nombre plus bas par un 0
    return [x for x in tokens if len(x) > 3 ]

In [ ]:
def remove_stopwords(tokens):
# def remove_stopwords(text):
# la fonction va retirer du texte les termes qui sont trop courant pour avoir une signification à partir d'un dictionnaire général. 

    if LANGUE_CORPUS == 'FR':
        nlp = spacy.load('fr_core_news_sm')
    elif LANGUE_CORPUS == 'EN':
        nlp = spacy.load('en_core_web_sm')
    else :
        print("il y a une problème avec la langue - stopword")

    stopwords = nlp.Defaults.stop_words
    tokens = set(tokens)
    return list(tokens - stopwords)

In [ ]:
def return_articles(text):
# La fonction permet de restaurer les textes après le nettoyage
    return " ".join([word for word in text])

In [ ]:
def lemmatize(text):
# la lemmatisation ramène les mots au lemme c'est à dire l'expression générale sans accord de genre ou de nombre.
    if LANGUE_CORPUS == 'FR':
        nlp = spacy.load('fr_core_news_sm')
    elif LANGUE_CORPUS == 'EN':
        nlp = spacy.load('en_core_web_sm')
    else :
        print("il y a une problème avec la langue - stopword")
        
    tokens = nlp(text)
    text = " ".join([w.lemma_ for w in tokens])
    return text

In [ ]:
def get_bow_matrix(df):
    # les données sont mises en forme dans par sacs de mots (Bag Of Words)
    X = 0
    try:
        # vectorizer = CountVectorizer(min_df = 0.15, max_df = 0.9) 
        # - Paramètres proposé originalement par Toufik = 8 dimensions, pas bon. Essayer les variations, revue littérature sur la réduction des dimensions
        vectorizer = CountVectorizer()
        #print(list(df['lemma_words']))
        #print(len(list(df['lemma_words'])))
        X = vectorizer.fit_transform(list(df['lemma_words'])).toarray()
        #print(vectorizer.get_feature_names_out())
        #print(len(vectorizer.get_feature_names_out()))
        #print(len(X[0]))
        print('\n  >>  Transformation de données en BOW::    --    État :: '+str(u"\u2713"))
    except:
        print('\n  >>  Transformation de données en BOW ::    --    État :: échec')
    return (X, vectorizer.get_feature_names_out())

In [ ]:
def get_tfidf_matrix(bow_matrix):
  # Pondération avec une fonction de TF/IDF 
  # TF/IDF = Term frequency / Inverse document frequency afin de permettre aux mots trop ou trop peu fréquent de perdre de la valeur dans la matrice
    try:
        # Transformation de Sacs de mots BOW en une matrice TFIDF 
        transformer = TfidfTransformer()
        tfidf_matrix = transformer.fit_transform(bow_matrix).toarray()
        print(len(tfidf_matrix[0]))
        print(len(tfidf_matrix))
        print('\n  >>  Transformation de BOW en une matrice TFIDF ::    --    État: '+str(u"\u2713"))
    except:
        print('\n  >>  Transformation de BOW en une matrice TFIDF  ::    --    État: échec')
    return tfidf_matrix

In [ ]:
def find_optimal_clusters(data, max_k):

    
#    La fonction permet de trouver le nombre optimal de clusters à utiliser
#    et dessine le graphe correspondant a chaque nombre de clusters

    iters = range(2, max_k+1, 2)
    sse = []
    for k in iters:
        sse.append(MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20).fit(data).inertia_)
        print('{} clusters'.format(k)) 
    f, ax = plt.subplots(1, 1)
    ax.plot(iters, sse, marker='o')
    ax.set_xlabel('Centres de Clusters ')
    ax.set_xticks(iters)
    ax.set_xticklabels(iters)
    ax.set_ylabel('SSE')
    ax.set_title('SSE par Cluster ')

In [ ]:
def plot_tsne_pca(data, labels):
    """
    plot_tsne_pca permet la visualisation de 
    - Données haute dimensions gràce a la technique TSNE 
    - Composantes principales gràce au PCA
    --  Les paramètres
        @data : Les données à visualiser
        @labels: les étiquettes de clusters 
        @return: deux graphes, un des composantes principales 
        et l'autre TNSE pour visualiser les clusters
    """
    # Cette visualisation ne suffit pas pour évaluer et explorer les clusters, 
    # il faut le retour vers le corpus ajoutant une colonne au df original 
    # et on pourrait utiliser le mot le plus fréquent (les 10 mots) de ce cluster comme étiquette intelligible
    
    max_label = max(labels)
    max_items = np.random.choice(range(data.shape[0]), size=data.shape[0])
    pca = PCA(n_components=2).fit_transform(data[max_items,:])
    tsne = TSNE().fit_transform(PCA(n_components=2).fit_transform(data[max_items,:]))
    
    idx = np.random.choice(range(pca.shape[0]), size=840)
    label_subset = labels[max_items]
    label_subset = [cm.hsv(i/max_label) for i in label_subset[idx]]
    f, ax = plt.subplots(1, 2, figsize=(14, 5))
    ax[0].scatter(pca[idx, 0], pca[idx, 1], c=label_subset)
    ax[0].set_title('Clusters-PCA  Plot')

    ax[1].scatter(tsne[idx, 0], tsne[idx, 1], c=label_subset)
    ax[1].set_title('Clusters-TNSE Plot')

In [ ]:
def checkThems(text):
# fonction de recherche d'information par mot-clé
    words = text.split()
    themes_words = []
    for w in words:
        if STEMMER.stem(w) in ALIMENTS:
            themes_words.append(w)
            print("TESTTheme", themes_words)
    return themes_words

In [ ]:
def checkChamps(text):
    # fonction de recherche d'information par mot-clé 
    # combiner avec checkThems

    words = text.split()
    champsl = []
    for w in words:
        if STEMMER.stem(w) in TERMES:
            champsl.append(w)
            print("TESTChamps", champsl)
    return champsl

In [ ]:
def searchCatTree(text, df_synset):
   # La fonction searchCatTree permet de retourner le nom d'un ensemble de synonyme ou d'une hiérarchie de catégories"

    list_words = []
    words = text.split()
    for w in words:
        if STEMMER.stem(w) in df_synset[syn1]: ### en construction
            list_words.append(w)
            print("TESTCatTree", w)
    

    return(list_words)

In [ ]:
def scoreCatTree(list_words, df):

 #   La fonction searchCatTree permet de retourner le la valeur d'une valeur spécifique associé à une catégorisation. 
 #   Il faut faire compter les scores de chaque mots qui apparait, sommer les scores et faire la moyenne.


    list_score = []
    sum_score = []
    n_score = 0
    avr_score = 0.0
    

    return(list_score, sum_score, n_score, avr_score)


In [ ]:
def getNer(text):
    """
    La fonction getNer permet d'extraire les entités nommées 
    --  Les paramètres
        @Text : le texte à traiter  
        @Return: La liste des entités nommés par catégories

        # Esi : il faut voir à séparer les noms et prénoms = https://github.com/flairNLP/flair/blob/master/resources/docs/TUTORIAL_2_TAGGING.md
        # Esi : L'étiquette B-PER est associé au prénom et E-PER est associé au patronyme
    """
    ner_list_per =[]
    ner_list_loc =[]
    ner_list_org =[]
    ner_list_misc =[]
    ner_list_others =[]
    
    sentence = Sentence(text)
    tagger.predict(sentence)
    for entity in sentence.get_spans('ner'):
        label_entity = re.sub('\d','',str(entity.labels[0])).replace('(','').replace(')','')
        #display(label_entity)
        if label_entity == 'PER .':
            ner_list_per.append(re.sub('\d','',str(entity.tokens[0]).replace('Token: ','')).replace(' ',''))
        elif label_entity == 'LOC .':
            ner_list_loc.append(re.sub('\d','',str(entity.tokens[0]).replace('Token: ','')).replace(' ',''))
        elif label_entity == 'ORG .':
            ner_list_org.append(re.sub('\d','',str(entity.tokens[0]).replace('Token: ','')).replace(' ',''))
        elif label_entity == 'MISC .':
            ner_list_misc.append(re.sub('\d','',str(entity.tokens[0]).replace('Token: ','')).replace(' ',''))
        else:
            ner_list_others.append(re.sub('\d','',str(entity.tokens[0]).replace('Token: ','')).replace(' ',''))
        # ner_list.append(re.sub('\d','',str(entity.tokens[0]).replace('Token: ','')).replace(' ',''))
        # ner_labels.append(re.sub('\d','',str(entity.labels[0])).replace('(','').replace(')',''))
    return {'ner_list_per': ner_list_per, 'ner_list_loc': ner_list_loc, 'ner_list_org': ner_list_org, 'ner_list_misc': ner_list_misc, 'ner_list_others': ner_list_others }

MAIN
Rouler ces blocs de fonctions ne devrait pas être très long. 
Ici commence l'exécution de ces fonctions sur les textes et le temps de traitement peut varier considérablement selon l'ensemble de données utilisé. Il est suggéré de faire un premier test avec un corpus réduit à 100 entrées pour estimer le temps total de traitement du corpus. Le corpus doit contenir un minimum de 10 entrées (ou le nombre de cluster min) pour ne pas avoir d'erreur.

Si vous voulez sautez une fonction (par exemple le retrait des stop-words), il suffit de mettre le bloc en commentaire ou le sauter simplement à cette étape plutôt que plus haut dans le code ou les variables et fonctions sont définies.

In [1]:
# chargement de données (corpus)
# MAIN
print(PATH_CORPUS)
start_time = time.time()
data_origin = get_dataset(PATH_CORPUS)
data = merge_col(data_origin, LISTE_COLONNE)
data["Doc_ID"] = data_origin["Doc_ID"]
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))
display(data)


#chargement du synset
start_time = time.time()
synset = get_dataset(PATH_SYNSET)
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))
display(data)


NameError: name 'PATH_CORPUS' is not defined

## Pré-traitement

In [ ]:
# retrait de ponctuations
start_time = time.time()
data['removed_punc'] = data['line'].apply(lambda x: remove_punct(x))
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))
display(data)

In [ ]:
# tokenisation
start_time = time.time()
data['tokens'] = data['removed_punc'].apply(lambda txt : tokenize(txt))
# data.drop(['removed_punc'], axis=1, inplace=True)
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))
display(data)

In [ ]:
# Suppression des tokens dont la taille < 3
start_time = time.time()
data['larger_tokens'] = data['tokens'].apply(lambda x : remove_small_words(x))
# data.drop(['tokens'], axis=1, inplace=True)
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))

display(data)

In [ ]:
# Suppression des stop words
start_time = time.time()
data['clean_tokens'] = data['larger_tokens'].apply(lambda x : remove_stopwords(x))
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))

In [ ]:
# fusion des tokens 
start_time = time.time()
data['clean_text'] = data['clean_tokens'].apply(lambda x : return_articles(x))
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))

In [ ]:
# Lemmatisation 
start_time = time.time()
data['lemma_words'] = data['clean_text'].apply(lambda x : lemmatize(x))
# data.drop(['clean_text'], axis=1, inplace=True)
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))

## Génération de corpus pré-traité

In [ ]:
# Sauvegarder le corpus nettoyé dans un fichier excel
# à faire : réordonner les colonnes 
# + ajouter une variable globale de noms de colonnes du dataset original à ajouter aux sorties à titre de parametrage

data.to_excel('corpus_nettoyé.xlsx',index=False) 

In [ ]:
#pour reloader le copus lémmatisé et nettoyé sans refaire le prétraitement
#data = pd.read_excel('corpus_nettoyé.xlsx', usecols=col) 

## Vectorisation et Clustering des articles 

In [ ]:
# # Transformation de données en BOW- Chaine de traitement jusqu'à TF/IDF

# bow_matrix, columns_name = get_bow_matrix(data)
# data_bow=pd.DataFrame(bow_matrix, columns=columns_name)
# #print(bow_matrix)
# #print(columns_name)
# #display(data_bow)
# data_bow.to_excel('corpus_Bow_Matrix.xlsx',index=False)

In [ ]:
# # Transformation de BOW en une matrice TF/IDF
# tfidf_matrix = get_tfidf_matrix(bow_matrix)

# data_tfidf=pd.DataFrame(tfidf_matrix, columns=columns_name)
# print(tfidf_matrix)
# display(data_tfidf)
# data_tfidf.to_excel('corpus_TFIDF_Matrix.xlsx',index=False)


## Optimisation du calcul de nombre de clusters

In [ ]:
# # la recherche du nombre optimal de clusters à utiliser 
# find_optimal_clusters(tfidf_matrix, 20)

In [ ]:
# # Clustering
# # Ancient paramètres données par Toufik
# # clusters = MiniBatchKMeans(n_clusters=10, init_size=840, batch_size=200, random_state=20).fit_predict(tfidf_matrix)
# clusters = MiniBatchKMeans(n_clusters=10, random_state=20)

# clusters_tfidf = clusters.fit_predict(tfidf_matrix)
# data['cluster_tfidt'] = clusters_tfidf

# clusters_bow = clusters.fit_predict(bow_matrix)
# data['cluster_bow'] = clusters_bow

# # pour sortir le corpus avec les clusters afin de comparer les incrémentations.
# # data.to_excel('corpus_cluster.xlsx',index=False)

## Extractions et analyses

In [ ]:
#plot_tsne_pca(tfidf_matrix, clusters)

## Recherche des entités nommées

C'est le loin le traitement le plus lourds, il peut s'agir de plusieurs minutes par entrée pour ce traitement selon la taille du texte.

In [ ]:
# Chargement du Flair SequenceTagger
if LANGUE_CORPUS == 'FR':
    tagger = SequenceTagger.load("flair/ner-french")
elif LANGUE_CORPUS =='EN':
    tagger = SequenceTagger.load("ner")
else:
    print('il y a une problème avec la langue - tagger entités nommées')

In [ ]:
# Appel de la fonction reconnaissance des entités nommées = 3min pour 12rec

start_time = time.time()

# Il faut tester la qualité des résultats en fonction de la version de prétraitement : est-ce que les majuscules améliorent le nombre d'entités repérées ?
# la colonne : data[raw] contient la fusion des colonnes sélectionnées AVEC les majuscules 

new_test_df = data.apply(lambda x: getNer(x.raw), axis='columns', result_type='expand')

display(new_test_df)

data = pd.concat([data, new_test_df], axis='columns')

print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))

In [ ]:
# Recherche d'information à partir d'une liste de thèmes

start_time = time.time()
data['Aliments'] = data['lemma_words'].apply(lambda x: checkThems(x))
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))

In [ ]:
# Recherche d'information sur les termes 

start_time = time.time()
# data['Termes'] = data['line'].apply(lambda x: checkChamps(x))
data['Terme'] = data['lemma_words'].apply(lambda x: checkChamps(x))
print("\n  >> temps d\'execution : {:.2f} s".format(round(time.time() - start_time, 2)))

In [ ]:
#Appeler recherche hiérarchique
#Apply searchCatTree

#Générer l'indice de centralité
#Apply scoreCatTree


## EXPORT
Ce dernier bloc est l'export du fichier de donnée enrichi. Il peut être roulé en cours de l'exploration pour s'assurer de la qualité du prétraitement et n'a pas a être roulé à la toute fin du programme.


In [ ]:
# Sauvegarde du dataframe sous forme MS Excel 

data.to_excel('corpus_enrichi.xlsx')